In [19]:
import os
import subprocess
import time
import sys
import threading
import socket
from urllib.request import Request, urlopen
import re

!pip install bcrypt

# Function to install packages
def install_packages():
    subprocess.check_call([sys.executable, "-m", "pip", "install", "streamlit", "pyngrok", "pyjwt", "watchdog"])
print("Installing required packages...")
install_packages()

# Import after installation
from pyngrok import ngrok

# --- Create Streamlit Config for Dark Theme ---
os.makedirs(".streamlit", exist_ok=True)
config_toml = """
[theme]
base="dark"
primaryColor="#00F5FF"
backgroundColor="#041C32"
secondaryBackgroundColor="#06283D"
textColor="#FFFFFF"
font="sans serif"

[server]
headless = true
"""
with open(".streamlit/config.toml", "w") as f:
    f.write(config_toml)

print("Applied Neon Dark Theme configuration.")

Installing required packages...
Applied Neon Dark Theme configuration.


In [20]:
# --- Create the Streamlit App File ---
app_code = """
import streamlit as st
import jwt
import datetime
import sqlite3
import re
import os
import bcrypt

# ---------------- CONFIG ----------------
SECRET_KEY = "super_secret_key"
ALGORITHM = "HS256"
DB_NAME = "users.db"

st.set_page_config(
    page_title="Infosys Springboard",
    page_icon="🎓",
    layout="centered"
)


# ---------------- NEON CSS ----------------
st.markdown(\"\"\"
<style>

/* Main Background */
.stApp {
    background: linear-gradient(135deg, #041C32, #06283D);
}

/* Glass Card */
.neon-card {
    background: rgba(255,255,255,0.05);
    backdrop-filter: blur(15px);
    padding: 40px;
    border-radius: 20px;
    border: 1px solid rgba(0,245,255,0.5);
    box-shadow: 0 0 25px rgba(0,245,255,0.4);
    animation: fadeIn 0.8s ease-in-out;
}

/* Titles */
h1, h2, h3 {
    color: #00F5FF;
    text-align: center;
}

/* Inputs */
.stTextInput>div>div>input {
    background: transparent;
    border: none;
    border-bottom: 2px solid #00F5FF;
    color: white;
    font-size: 16px;
}

.stSelectbox>div>div {
    background: rgba(255,255,255,0.05);
    border: 1px solid #00F5FF;
    border-radius: 10px;
    color: white;
}

/* Buttons */
.stButton>button {
    width: 100%;
    border-radius: 30px;
    padding: 12px;
    font-weight: bold;
    border: none;
    background: linear-gradient(90deg, #00F5FF, #00C9A7);
    color: black;
    transition: 0.3s ease;
    box-shadow: 0 0 15px rgba(0,245,255,0.6);
}

.stButton>button:hover {
    transform: scale(1.05);
    box-shadow: 0 0 25px rgba(0,245,255,1);
}

/* Sidebar */
[data-testid="stSidebar"] {
    background: #031926;
}

/* Animation */
@keyframes fadeIn {
    from {opacity: 0; transform: translateY(20px);}
    to {opacity: 1; transform: translateY(0);}
}

</style>
\"\"\", unsafe_allow_html=True)

# ---------------- DATABASE ----------------
conn = sqlite3.connect(DB_NAME, check_same_thread=False)
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS users (
    username TEXT,
    email TEXT PRIMARY KEY,
    password BLOB,
    security_question TEXT,
    security_answer TEXT
)
''')
conn.commit()

# ---------------- JWT ----------------
def create_token(data):
    data["exp"] = datetime.datetime.utcnow() + datetime.timedelta(hours=1)
    return jwt.encode(data, SECRET_KEY, algorithm=ALGORITHM)

# ---------------- VALIDATION ----------------
def valid_email(email):
    pattern = r'^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$'
    return re.match(pattern, email)

def valid_password(password):
    pattern = r'^(?=.*[a-z])(?=.*[A-Z])(?=.*\\d)(?=.*[@$!%*?&])[A-Za-z\\d@$!%*?&]{8,}$'
    return re.match(pattern, password)

def password_strength(password):
    score = 0
    if len(password) >= 8: score += 1
    if re.search(r'[A-Z]', password): score += 1
    if re.search(r'[a-z]', password): score += 1
    if re.search(r'\\d', password): score += 1
    if re.search(r'[@$!%*?&]', password): score += 1
    return score

# ---------------- SESSION ----------------
if "page" not in st.session_state:
    st.session_state.page = "login"

if "token" not in st.session_state:
    st.session_state.token = None

# ---------------- SIGNUP ----------------
def signup():
    st.markdown(\"\"\"
    <div class="neon-card" style="
        display:flex;
        justify-content:center;
        align-items:center;
        text-align:center;
        font-size:28px;
        font-weight:600;
        letter-spacing:0.5px;
        color:#ffffff;
        text-shadow:0 0 12px rgba(0,245,255,0.8);
    ">
        Welcome to Infosys Springboard Internship
    </div>
    \"\"\", unsafe_allow_html=True)
    st.title("Create Account")

    questions = [
        "What is your pet name?",
        "What is your mother’s maiden name?",
        "What is your favorite teacher?",
        "What was your first school name?",
        "What is your favorite food?"
    ]

    with st.form("signup_form"):
        username = st.text_input("Username")
        email = st.text_input("Email")
        password = st.text_input("Password", type="password")
        confirm = st.text_input("Confirm Password", type="password")
        question = st.selectbox("Security Question", questions)
        answer = st.text_input("Security Answer")
        submit = st.form_submit_button("Signup")

        # Show password strength
        if password:
            score = password_strength(password)
            st.text(f"Password strength: {score}/5")

        if submit:
            if not all([username,email,password,confirm,answer]):
                st.error("All fields are mandatory")
            elif not valid_email(email):
                st.error("Invalid Email format")
            elif not valid_password(password):
                st.error("Password must be strong (8+ chars, uppercase, lowercase, number, special char)")
            elif password != confirm:
                st.error("Passwords do not match")
            else:
                cursor.execute("SELECT email FROM users WHERE email=?", (email,))
                existing_user = cursor.fetchone()
                if existing_user:
                    st.error("Email already registered")
                else:
                    hashed_pw = bcrypt.hashpw(password.encode(), bcrypt.gensalt())
                    cursor.execute(
                        "INSERT INTO users VALUES (?,?,?,?,?)",
                        (username,email,hashed_pw,question,answer)
                    )
                    conn.commit()
                    st.success("Account created successfully 🎉")

    if st.button("Go to Login"):
        st.session_state.page="login"
        st.rerun()

    st.markdown('</div>', unsafe_allow_html=True)

# ---------------- LOGIN ----------------
def login():
    st.markdown(\"\"\"
    <div class="neon-card" style="
        display:flex;
        justify-content:center;
        align-items:center;
        text-align:center;
        font-size:28px;
        font-weight:600;
        letter-spacing:0.5px;
        color:#ffffff;
        text-shadow:0 0 12px rgba(0,245,255,0.8);
    ">
        Welcome to Infosys Springboard Internship
    </div>
    \"\"\", unsafe_allow_html=True)
    st.title("Login")

    with st.form("login_form"):
        email = st.text_input("Email")
        password = st.text_input("Password", type="password")
        submit = st.form_submit_button("Login")

        if submit:
            cursor.execute("SELECT username,password FROM users WHERE email=?", (email,))
            data = cursor.fetchone()
            if data and bcrypt.checkpw(password.encode(), data[1]):
                st.session_state.token=create_token({"email":email})
                st.session_state.page="dashboard"
                st.rerun()
            else:
                st.error("Invalid credentials")

    col1, col2 = st.columns(2)
    with col1:
        if st.button("Forgot Password"):
            st.session_state.page="forgot"
            st.rerun()
    with col2:
        if st.button("Create Account"):
            st.session_state.page="signup"
            st.rerun()

    st.markdown('</div>', unsafe_allow_html=True)

# ---------------- DASHBOARD ----------------
def dashboard():
    payload = jwt.decode(st.session_state.token, SECRET_KEY, algorithms=[ALGORITHM])
    email = payload["email"]

    cursor.execute("SELECT username FROM users WHERE email=?", (email,))
    username = cursor.fetchone()[0]

    st.markdown(\"\"\"
    <div class="neon-card" style="
        display:flex;
        justify-content:center;
        align-items:center;
        text-align:center;
        font-size:28px;
        font-weight:600;
        letter-spacing:0.5px;
        color:#ffffff;
        text-shadow:0 0 12px rgba(0,245,255,0.8);
    ">
        Welcome to Infosys Springboard Internship
    </div>
    \"\"\", unsafe_allow_html=True)
    st.title("Dashboard")
    st.success(f"Welcome back, {username} 🚀")

    if st.button("Logout"):
        st.session_state.token=None
        st.session_state.page="login"
        st.rerun()

    st.markdown('</div>', unsafe_allow_html=True)

# ---------------- FORGOT PASSWORD ----------------
def forgot_password():
    st.markdown(\"\"\"
    <div class="neon-card" style="
        display:flex;
        justify-content:center;
        align-items:center;
        text-align:center;
        font-size:28px;
        font-weight:600;
        letter-spacing:0.5px;
        color:#ffffff;
        text-shadow:0 0 12px rgba(0,245,255,0.8);
    ">
        Welcome to Infosys Springboard Internship
    </div>
    \"\"\", unsafe_allow_html=True)
    st.title("Forgot Password")

    email = st.text_input("Enter your registered Email")

    if st.button("Verify Email"):
        cursor.execute("SELECT security_question FROM users WHERE email=?", (email,))
        data = cursor.fetchone()
        if data:
            st.session_state.reset_email=email
            st.session_state.security_question=data[0]
            st.success("Email verified")
        else:
            st.error("Email not found")

    if "security_question" in st.session_state:
        st.write(st.session_state.security_question)
        answer = st.text_input("Enter your Answer")

        if st.button("Verify Answer"):
            cursor.execute("SELECT security_answer FROM users WHERE email=?",
                           (st.session_state.reset_email,))
            real_answer = cursor.fetchone()
            if real_answer and answer == real_answer[0]:
                st.session_state.answer_verified=True
                st.success("Verified")
            else:
                st.error("Wrong answer")

    if st.session_state.get("answer_verified"):
        new_pass = st.text_input("Enter New Password", type="password")
        confirm_pass = st.text_input("Confirm New Password", type="password")

        if st.button("Update Password"):
            if not valid_password(new_pass):
                st.error("Weak password")
            elif new_pass != confirm_pass:
                st.error("Passwords do not match")
            else:
                hashed_pw = bcrypt.hashpw(new_pass.encode(), bcrypt.gensalt())
                cursor.execute("UPDATE users SET password=? WHERE email=?",
                               (hashed_pw , st.session_state.reset_email))
                conn.commit()
                st.success("Password updated successfully 🎉")
                st.session_state.page="login"
                st.rerun()

    if st.button("Back to Login"):
        st.session_state.page="login"
        st.rerun()

    st.markdown('</div>', unsafe_allow_html=True)

# ---------------- ROUTING ----------------
if st.session_state.token:
    dashboard()
else:
    if st.session_state.page=="signup":
        signup()
    elif st.session_state.page=="forgot":
        forgot_password()
    else:
        login()
"""

with open("app.py","w") as f:
    f.write(app_code)

print("✅ Infosys SpringBoard app.py")


✅ Infosys SpringBoard app.py


<>:119: SyntaxWarning: invalid escape sequence '\.'
<>:119: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipython-input-1860962776.py:119: SyntaxWarning: invalid escape sequence '\.'
  pattern = r'^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$'


In [ ]:
# --- Wait for Streamlit to Start ---
def wait_for_streamlit(port=8501, timeout=30):
    start_time = time.time()
    while time.time() - start_time < timeout:
        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(1)
            result = sock.connect_ex(('localhost', port))
            if result == 0:
                sock.close()
                return True
            sock.close()
        except Exception:
            pass
        time.sleep(1)
    return False
# --- Ngrok Setup ---
print("\nTo access the app, you need an Ngrok Authtoken.")
print("Get it from: https://dashboard.ngrok.com/get-started/your-authtoken")
authtoken = input("Enter your Ngrok Authtoken: ").strip()
if authtoken:
    ngrok.set_auth_token(authtoken)

    # Kill any existing ngrok process
    os.system("pkill ngrok")
    os.system("pkill streamlit")

    # Run Streamlit in the background FIRST
    print("Starting Streamlit...")
    # Using Subprocess.Popen to run in background
    # Redirecting output to /dev/null to keep cell clean
    process = subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501", "--server.address", "localhost"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # Wait for it to be ready
    if wait_for_streamlit():
        print("Streamlit is active! Connecting Ngrok...")
        # Open a tunnel to the streamlit port 8501
        try:
            public_url = ngrok.connect(8501).public_url
            print(f"\n🚀 Streamlit App is running!")
            print(f"👉 Public URL: {public_url}")
            print("\n(Click the URL above to open the app)")

            # Keep main thread alive
            try:
                # Keep checking if process is alive
                while process.poll() is None:
                    time.sleep(1)
            except KeyboardInterrupt:
                print("Stopping...")
                ngrok.disconnect(public_url)
                process.terminate()
        except Exception as e:
            print(f"Ngrok connection failed: {e}")
            process.terminate()
    else:
        print("Error: Streamlit failed to start in time.")
        process.terminate()
else:
    print("Ngrok Authtoken is required to expose the app publicly.")


To access the app, you need an Ngrok Authtoken.
Get it from: https://dashboard.ngrok.com/get-started/your-authtoken
